## OpenAltimetry IceSat 2 Workflows

### Overview

This notebook uses [ATL03](https://nsidc.org/data/ATL03/versions/1) data from the IceSat 2 mission via the OpenAltimetry API for subsetting. This API enable users to select a particular region of interest and get pre-processed photon data wihtout the need to deal with big HDF files.

### Instructions

This notebook takes some parameters from OA and some have to be manualy extracted, the bounding box, available dates and ground track information are availabe via the OpenAltimetry web app.

In [ ]:
# First we plot repeated pass over the same groundtrack.

import numpy as np
import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import requests
import pyproj
from urllib import parse 
import math
import plotly.graph_objects as go
from plotly.offline import iplot


### Parameters ###
OA_BBOX = 'minx=-41.3228&miny=-78.9439&maxx=-39.0422&maxy=-78.4981'
OA_TRACK = '314'
OA_DATES = ['2018-10-19', '2019-01-18']
# PLOTTED_BEAMS = [1,2,3,4,5,6]
PLOTTED_BEAMS = [1,2]
# PHOTONS = ['Noise', 'Low', 'Medium', 'High']
PHOTONS = ['Medium', 'High']
### Parameters end ###


# We populate a list with photon data using the OpenAltimetry API
df_beams = []
oa_plots = []
COLOR_SCALES = ['Viridis', 'Jet', 'Cividis', 'Electric']
# PHOTONS = ['Noise', 'Low', 'Medium', 'High']


def getPhotonData(date, track):
    OA_URL = 'https://openaltimetry.org/data/icesat2/getPhotonData?' + OA_BBOX + '&trackId=' + track + '&date=' + date
    series = []
    for beam in PLOTTED_BEAMS:
        b_url = OA_URL + '&beam=' + str(beam)
        r = requests.get(b_url)
        print('Requesting Beam {} for date: {}'.format(beam, date))
        data = r.json()
        series.append(data)
    return series

# Main loop for multiple dates
color_index = -1
for td in OA_DATES:
    photon_cloud = getPhotonData(td, OA_TRACK)
    color_index +=1
    t_index = 0
    for track in photon_cloud:
        for photons in track['series']:
            if any(word in photons['name'] for word in PHOTONS):
                series = []
                for p in photons['data']:
                    series.append({
                        'lat': p[0],
                        'lon': p[2],
                        'h': p[1],
                        'date': td,
                        'beam': PLOTTED_BEAMS[t_index]
                    })
                if (len(series) > 0):
                    df = pd.DataFrame.from_dict(series)
                    df.name = td + ' beam ' + str(PLOTTED_BEAMS[t_index]) + ' ' + photons['name']
                    df.colorscale = COLOR_SCALES[color_index]
                    df_beams.append(df)
        t_index +=1

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'], mode='markers',
                                    marker=dict(
                                        size=1,
                                        color=df['h'],
                                        colorscale=df.colorscale,   # choose a colorscale
                                        opacity=0.8
                                    )
                                )
                   )

In [ ]:

layout = go.Layout(
    width=900,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

fig = go.Figure(data=oa_plots, layout=layout)  

iplot(fig)

### Now let's do some science with the data we already have in Pandas

In [ ]:
# Elevation histofgrams
series_df = pd.concat(df_beams)

b_df = pd.DataFrame(series_df[(series_df.date == '2018-10-19') & (series_df.beam == 1)])
a_df = pd.DataFrame(series_df[(series_df.date == '2019-01-18') & (series_df.beam == 2)])

print(b_df.head())
print(a_df.head())



### Now we explore some cross-tracks

In [ ]:
import numpy as np
import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import requests
import pyproj
from urllib import parse 
import math
import plotly.graph_objects as go
from plotly.offline import iplot


# Paste the OpenAltimetry reference values here for repeated pass
OA_BBOX = 'minx=-67.9616&miny=-20.9&maxx=-67.4473&maxy=-19.8'
OA_TRACK_1 = {'name': '270', 'date': '2019-04-15', 'params': '&trackId=270&date=2019-04-15' }
OA_TRACK_2 = {'name': '826', 'date': '2018-11-21', 'params': '&trackId=826&date=2018-11-21' }
PLOTTED_BEAMS = [1,2,3,4,5,6]
PHOTONS = ['Medium', 'High']


# We populate a list with photon data using the OpenAltimetry API
df_beams = []
oa_plots = []
OA_TRACKS = [OA_TRACK_1, OA_TRACK_2]
COLOR_SCALES = ['Viridis', 'Electric']
# PHOTONS = ['Noise', 'Low', 'Medium', 'High']


def getPhotonDataCrossTrack(track_params):
    OA_URL = 'https://openaltimetry.org/data/icesat2/getPhotonData?client=jupyter&' + OA_BBOX + track_params
    series = []
    for beam in PLOTTED_BEAMS:
        b_url = OA_URL + '&beam=' + str(beam)
        print('Requesting Beam {}'.format(beam))
        r = requests.get(b_url, timeout=60)
        data = r.json()
        series.append(data)
        print('....data fetched')
    return series

# Main loop for multiple dates
color_index = -1
for tr in OA_TRACKS:
    photon_cloud = getPhotonDataCrossTrack(tr['params'])
    color_index +=1
    t_index = 0
    for track in photon_cloud:
        for photons in track['series']:
            if any(word in photons['name'] for word in PHOTONS):
                series = []
                for p in photons['data']:
                    series.append({
                        'lat': p[0],
                        'lon': p[2],
                        'h': p[1],
                        'date': tr['date'],
                        'beam': PLOTTED_BEAMS[t_index]
                    })
                if (len(series) > 0):
                    df = pd.DataFrame.from_dict(series)
                    df.name = 'Track [' + tr['name'] + '] beam ' + str(PLOTTED_BEAMS[t_index]) + ' ' + photons['name']
                    df.colorscale = COLOR_SCALES[color_index]
                    df_beams.append(df)
        t_index +=1

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'], mode='markers',
                                    marker=dict(
                                        size=1,
                                        color=df['h'],
                                        colorscale=df.colorscale,   # choose a colorscale
                                        opacity=0.8
                                    )
                                )
                   )

In [ ]:

layout = go.Layout(
    width=900,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

fig = go.Figure(data=oa_plots, layout=layout)  

iplot(fig)